# Mortality Prediction


@References : Soenksen, L.R., Ma, Y., Zeng, C. et al. Integrated multimodal artificial intelligence framework for healthcare applications. npj Digit. Med. 5, 149 (2022). https://doi.org/10.1038/s41746-022-00689-4

In this notebook, the task is to predict 48 hour mortality using the CSV embeddings file



## Introduction


The goal of this part of the study is to build models to predict the probability that a patient will expire during the next 48 h as a binary classification problem: expired ≤48 h (1) or otherwise (0). In the case of a patient whose hospital exit status is not expiration, the class label is set to 0. A patient can acquire different target class labels at different time points during their stay due to changes in status and proximity to the discharge or time of death. Similar to the length-of-stay modeling, each sample in this predictive task corresponds to a single patient-admission EHR time point where an X-ray image was obtained (N = 45,050).


#### Imports

In [1]:
import os
os.chdir('../')

from pandas import read_csv

from src.data import constants
from src.data.dataset import HAIMDataset
from src.evaluation.pycaret_evaluator import PyCaretEvaluator
from src.utils.metric_scores import *

#### Read data from local source



In [2]:
df = read_csv(constants.FILE_DF, nrows=constants.N_DATA)

#### Create a custom dataset for the HAIM experiment


Build the target column for the task at hand, set the dataset specificities:  the ``haim_id`` as a ``global_id``, use all sources for prediction

In [3]:
dataset = HAIMDataset(df,  
                      constants.ALL_PREDICTORS, 
                      constants.ALL_MODALITIES, 
                      constants.MORTALITY, 
                      constants.IMG_ID, 
                      constants.GLOBAL_ID)

#### Set hyper-parameters

In [4]:
# Define the grid oh hyper-parameters for the tuning
grid_hps = {'max_depth': [5, 6, 7, 8],
            'n_estimators': [200, 300],
            'learning_rate': [0.3, 0.1, 0.05],
            }

### Model training and predictions using an XGBClassifier model with GridSearchCV and Hyperparameters optimization


The goal of this section of the notebook is to compute the following metrics:

``ACCURACY_SCORE, BALANCED_ACCURACY_SCORE, SENSITIVITY, SPECIFICITY, AUC, BRIER SCORE, BINARY CROSS-ENTROPY``


The
hyperparameter combinations of individual XGBoost models were
selected within each training loop using a ``fivefold cross-validated
grid search`` on the training set (80%). This XGBoost ``tuning process``
selected the ``maximum depth of the trees (5–8)``, the number of
``estimators (200 or 300)``, and the ``learning rate (0.05, 0.1, 0.3)``
according to the parameter value combination leading to the
highest observed AUROC within the training loop 


As mentioned previously, all XGBoost models were trained ``five times with five different data splits`` to repeat the
experiments and compute average metrics 


```Refer to page 8 of study``` : https://doi.org/10.1038/s41746-022-00689-4

In [5]:
# Initialize the PyCaret Evaluator
evaluator = PyCaretEvaluator(dataset=dataset, target="Mortality", experiment_name="CP_Mortality", filepath="./results/mortality")

# Model training
evaluator.run_experiment(
    train_size=0.8,
    fold=5,
    fold_strategy='kfold',
    outer_fold=5,
    outer_strategy='kfold',
    session_id=42,
    model='xgboost',
    optimize='AUC',
    custom_grid=grid_hps
)

2024-10-04 13:54:33,961	INFO worker.py:1777 -- Started a local Ray instance. View the dashboard at http://127.0.0.1:8265 


(run_fold pid=631326) Outer fold 1


(raylet) Spilled 3869 MiB, 2 objects, write throughput 589 MiB/s. Set RAY_verbose_spill_logs=0 to disable this message.
(raylet) Spilled 7738 MiB, 4 objects, write throughput 583 MiB/s.
(raylet) Spilled 11607 MiB, 5 objects, write throughput 579 MiB/s.
Processing:   0%|          | 0/4 [00:00<?, ?it/s]


(run_fold pid=631326) Configuring PyCaret for outer fold 1


Processing:  75%|███████▌  | 3/4 [13:23<04:57, 297.24s/it]
                                                          


...

(run_fold pid=631326) Configuring PyCaret for outer fold 5


Processing:  75%|███████▌  | 3/4 [12:36<04:39, 279.90s/it]
                                                          


(run_fold pid=631326)       Accuracy     AUC  Recall   Prec.      F1   Kappa     MCC
(run_fold pid=631326) Fold                                                          
(run_fold pid=631326) 0       0.9811  0.9537  0.3636  0.9375  0.5240  0.5163  0.5775
(run_fold pid=631326) 1       0.9849  0.9607  0.4345  0.9265  0.5915  0.5849  0.6289
(run_fold pid=631326) 2       0.9807  0.9553  0.3765  0.9275  0.5356  0.5275  0.5843
(run_fold pid=631326) 3       0.9811  0.9491  0.3681  0.9091  0.5240  0.5161  0.5719
(run_fold pid=631326) 4       0.9837  0.9530  0.4214  0.9710  0.5877  0.5807  0.6341
(run_fold pid=631326) Mean    0.9823  0.9544  0.3928  0.9343  0.5526  0.5451  0.5993
(run_fold pid=631326) Std     0.0017  0.0038  0.0293  0.0205  0.0306  0.0311  0.0266
(run_fold pid=631326) Tuning hyperparameters for model xgboost with custom grid using grid search
(run_fold pid=631326) Transformation Pipeline and Model Successfully Saved
(run_fold pid=631326)                        Model  Accuracy  

(raylet) [2024-10-07 08:24:39,646 E 631211 631211] (raylet) node_manager.cc:3065: 1 Workers (tasks / actors) killed due to memory pressure (OOM), 0 Workers crashed due to other reasons at node (ID: 4a76051cba771f3e77cb11a71acab1dc5b4773f9ce3f3946b8921878, IP: 10.44.86.85) over the last time period. To see more information about the Workers killed on this node, use `ray logs raylet.out -ip 10.44.86.85`
(raylet) 
(raylet) Refer to the documentation on how to address the out of memory issue: https://docs.ray.io/en/latest/ray-core/scheduling/ray-oom-prevention.html. Consider provisioning more memory on this node or reducing task parallelism by requesting more CPUs per task. To adjust the kill threshold, set the environment variable `RAY_memory_usage_threshold` when starting Ray. To disable worker killing, set the environment variable `RAY_memory_monitor_refresh_ms` to zero.


Final metrics table:
     Metric     Mean   Std Dev
0  Accuracy  0.98066  0.001385
1       AUC  0.96012  0.004346
2    Recall  0.35174  0.034586
3     Prec.  0.94114  0.021459
4        F1  0.51146  0.038680
5     Kappa  0.50372  0.038772
6       MCC  0.56860  0.032578
